# Data Exploration Notebook

This notebook covers the initial exploration of financial data for alpha factor generation and portfolio optimization.

## What it covers:

1. Pulling historical price and fundamentals data from OpenBB for your chosen universe.
2. Inspecting coverage: date ranges, missing values, ticker consistency.
3. Basic descriptive stats: average returns, volatility, sector breakdown.
4. Visual checks: price charts, return histograms, correlation heatmaps.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from openbb import obb

# Add src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import src functions
from src.data.loaders import get_equity_data
from src.features.fundamental import compute_financial_ratios
from src.features.technical import add_returns_column

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Pulling historical price and fundamentals data from OpenBB for your chosen universe.

In [ ]:
# Define the investment universe (example: large-cap US stocks)
universe = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX']
start_date = '2020-01-01'

# Pull historical price data for each ticker
price_data = {}
for ticker in universe:
    df = get_equity_data(ticker, start_date)
    if not df.empty:
        price_data[ticker] = df
        print(f"Pulled price data for {ticker}: {len(df)} rows")
    else:
        print(f"Failed to pull price data for {ticker}")

# Combine price data into a single DataFrame with multi-index
if price_data:
    combined_prices = pd.concat(price_data, axis=0, keys=price_data.keys())
    combined_prices.index.names = ['Ticker', 'Date']
    print(f"Combined price data shape: {combined_prices.shape}")
    print(combined_prices.head())
else:
    print("No price data pulled.")

In [ ]:
# Pull fundamentals data (example: balance sheet data)
fundamentals_data = {}
for ticker in universe:
    try:
        data = obb.equity.fundamentals.balance_sheet(ticker, provider="yfinance")
        df = data.to_df()
        if not df.empty:
            fundamentals_data[ticker] = df
            print(f"Pulled fundamentals for {ticker}: {df.shape}")
        else:
            print(f"No fundamentals data for {ticker}")
    except Exception as e:
        print(f"Error pulling fundamentals for {ticker}: {e}")

# For simplicity, we'll use the latest fundamentals data
latest_fundamentals = {}
for ticker, df in fundamentals_data.items():
    if not df.empty:
        latest = df.iloc[-1]  # Latest available data
        latest_fundamentals[ticker] = latest

fundamentals_df = pd.DataFrame.from_dict(latest_fundamentals, orient='index')
print(f"Fundamentals data shape: {fundamentals_df.shape}")
print(fundamentals_df.head())

In [ ]:
# Compute financial ratios using src/features/fundamental.py
# Note: This assumes fundamentals_df has columns like 'Price', 'Earnings', etc.
# In practice, you may need to map OpenBB columns to expected names.
# For demonstration, we'll create a sample mapping.

# Assuming fundamentals_df has relevant columns; if not, this will need adjustment
if not fundamentals_df.empty:
    # Add price data to fundamentals (using latest close price)
    for ticker in fundamentals_df.index:
        if ticker in price_data and not price_data[ticker].empty:
            latest_price = price_data[ticker]['Close'].iloc[-1]
            fundamentals_df.loc[ticker, 'Price'] = latest_price
    
    # Rename columns to match expected names (this is illustrative)
    column_mapping = {
        'Total Assets': 'BookValue',  # Approximate
        'Net Income': 'Earnings',
        'Total Debt': 'Debt',
        'Total Equity': 'Equity',
        'Market Capitalization': 'MarketCap'
    }
    fundamentals_df = fundamentals_df.rename(columns=column_mapping)
    
    # Compute ratios
    ratios_df = compute_financial_ratios(fundamentals_df)
    print("Computed financial ratios:")
    print(ratios_df[['P/E', 'P/B', 'D/E']].head())
else:
    print("No fundamentals data to compute ratios.")

## 2. Inspecting coverage: date ranges, missing values, ticker consistency.

In [ ]:
# Inspect date ranges for each ticker
print("Date range coverage:")
for ticker, df in price_data.items():
    if not df.empty:
        start = df.index.min()
        end = df.index.max()
        count = len(df)
        print(f"{ticker}: {start} to {end} ({count} observations)")
    else:
        print(f"{ticker}: No data")

# Check for missing values
print("\nMissing values summary:")
if 'combined_prices' in locals():
    missing_summary = combined_prices.isnull().sum()
    print(missing_summary)
    
    # Percentage of missing values
    missing_pct = (missing_summary / len(combined_prices)) * 100
    print("\nMissing values percentage:")
    print(missing_pct)

# Ticker consistency check
print("\nTicker consistency:")
tickers_with_data = [ticker for ticker, df in price_data.items() if not df.empty]
print(f"Tickers with data: {tickers_with_data}")
print(f"Total tickers in universe: {len(universe)}")
print(f"Tickers with data: {len(tickers_with_data)}")

## 3. Basic descriptive stats: average returns, volatility, sector breakdown.

In [ ]:
# Compute returns using src/features/technical.py
returns_data = {}
for ticker, df in price_data.items():
    if not df.empty:
        df_with_returns = add_returns_column(df.copy())
        returns_data[ticker] = df_with_returns

# Combine returns
if returns_data:
    combined_returns = pd.concat({ticker: df['Returns'] for ticker, df in returns_data.items()}, axis=1)
    combined_returns.columns = returns_data.keys()
    
    # Basic stats
    print("Average daily returns:")
    avg_returns = combined_returns.mean()
    print(avg_returns)
    
    print("\nAnnualized volatility (assuming 252 trading days):")
    volatility = combined_returns.std() * np.sqrt(252)
    print(volatility)
    
    print("\nDescriptive statistics:")
    print(combined_returns.describe())
else:
    print("No returns data to analyze.")

In [ ]:
# Sector breakdown
# Pull sector information using OpenBB
sector_data = {}
for ticker in universe:
    try:
        profile = obb.equity.profile(ticker, provider="yfinance")
        sector = profile.to_df().get('sector', 'Unknown')
        if isinstance(sector, pd.Series):
            sector = sector.iloc[0] if not sector.empty else 'Unknown'
        sector_data[ticker] = sector
    except Exception as e:
        sector_data[ticker] = 'Unknown'
        print(f"Error getting sector for {ticker}: {e}")

sector_df = pd.DataFrame.from_dict(sector_data, orient='index', columns=['Sector'])
print("Sector breakdown:")
sector_counts = sector_df['Sector'].value_counts()
print(sector_counts)

# Plot sector breakdown
plt.figure(figsize=(10, 6))
sector_counts.plot(kind='bar')
plt.title('Sector Breakdown of Universe')
plt.xlabel('Sector')
plt.ylabel('Number of Stocks')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Visual checks: price charts, return histograms, correlation heatmaps.

In [ ]:
# Price charts
plt.figure(figsize=(14, 8))
for ticker, df in price_data.items():
    if not df.empty:
        plt.plot(df.index, df['Close'], label=ticker)
plt.title('Historical Price Charts')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

In [ ]:
# Return histograms
if 'combined_returns' in locals():
    plt.figure(figsize=(12, 8))
    for column in combined_returns.columns:
        plt.hist(combined_returns[column].dropna(), bins=50, alpha=0.5, label=column)
    plt.title('Return Histograms')
    plt.xlabel('Daily Return')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
else:
    print("No returns data for histograms.")

In [ ]:
# Correlation heatmap
if 'combined_returns' in locals():
    correlation_matrix = combined_returns.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title('Return Correlation Heatmap')
    plt.show()
else:
    print("No returns data for correlation heatmap.")